# A Traditional Data Science Approach

Another thing that came to my mind on seeing game replays is that I can use this for training.([First thing that came to mind is here](https://www.kaggle.com/arunprathap/interpreting-replay-json)) I am setting aside RL agent level training which is already covered in other notebooks and thinking intuitively which of the fields in given data actually matters and how I can approach the game from a data science perspective.

## Get the JSON

Though I have used a sample JSON for this notebook, the following link can help you get more JSON.

[Notebook](https://www.kaggle.com/robga/google-football-episode-scraper) of [@robga](https://www.kaggle.com/robga) and the [dataset](https://www.kaggle.com/robga/gfootball001) used in his notebook.

## Import basic Libaries & Modules

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Enter URL of the game of interest

In [ ]:
url = 'https://www.kaggleusercontent.com/episodes/3634538.json'

## Load JSON from URL

In [ ]:
import urllib, json

response = urllib.request.urlopen(url)
data = json.loads(response.read())

### Format Step Data

This consists of same data (for 3002 steps) mirrored appropriately for the left and right players, and giving action, active player and sticky action fields which differ between left and right players.

Refer [link](https://github.com/google-research/football/blob/master/gfootball/doc/observation.md) for details on how to interpret these values. The fields that you may need to use may depend on your specific implementation. I am just giving an outline assuming you are using all fields. Note that you get double data with each game since left and right player data can be used for training. If you game plan depends on steps, consider adding step information also to your data.

The metric to be used should be something that gives insight in to how effective was this action and I am yet to think about it.

First index is for step(0 to 3001) and second for player(0 or 1)

Lets see a sample for step 1000.

In [ ]:
data['steps'][1000][0]

In [ ]:
data['steps'][1000][1]

The action corresponds to our output for training along with appropriate metric that determines how effective this action was.

Lets see which are the input features we need. The player that we are controlling is the most important thing. So 'left team roles' and 'left team tired factor' corresponding to 'active' player ('designated' is redundant since we are controlling only one player) is important. 'left team active' and 'left team yellow cards' corresponding to 'active' player also matter if you are planning to work out some offensive strategies. 'sticky actions' matter since that is part of the ongoing strategy of the team.

The ball is the big thing in the game and definitely 'ball' position, 'ball rotation' and 'ball direction' matters. Since our active player is always the one nearest to the ball, it may be beneficial to have a feature for distance between active player and ball as well. You will have to fit all this within the three laws of motion framework to get things moving in the game.

Nothing matters if the ball is unclaimed or in possession of the opposite team throughout the game. You need possession of ball for atleast as many steps as the number of goals you want (assuming you are the master player who gets a goal every time your feet touches the ball). So 'ball owned team', 'ball owned player' matters as well.

To know where your enemy is gives you the tactical advantage. You dont want to have a shot at goal post when you have enemy players on the strike side who can take possession of the ball. A high pass may save you the game. So there comes the right team positions, direction, tired factor, etc.

How good are you without having friends around. It's always effective to have 11 sets of feet in the ground than one to cover more ground. You can always pass the ball around the field waiting for opportunities or just fiddle around the field if you know where your friends are located. So the left player position, direction, tired factor, etc also plays a role in determining my action.

The 'game mode' is the next big thing as you don't want to worry about where your enemy is or you team mates are in atleast some of the modes.

Hope you liked my approach of intuitively going though the observation fields.

### EOF